In [2]:
# import the libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from tqdm import tqdm
from scipy.optimize import minimize
from hurst import compute_Hc
import warnings
warnings.filterwarnings("ignore")

In [3]:
def rsi(value, neutral, window):
	"""
	Input:
	:param value: (type dataframe pandas): Entry values of the stock
	:param neutral:(float) Value of the neutrality, i.e No Action Zone
	:param window: (float) rolling period for RSI calculation

	Output: The function gives the returns of RSI strategy
	:return:
	"""
	# Print error is no Adj Close column is not present
	if "Adj Close" not in value.columns:
		ValueError("Error: No Adj Close column is present")

	value['rsi'] = ta.momentum.RSIIndicator(value['Adj Close'], window=window).rsi()

	"""Long Buy signal"""
	overbuy = 70
	neutral_buy = 50 + neutral

	# Put nan values for te signal long columns
	value['signal_long'] = np.nan
	value['yesterday_rsi'] = value['rsi'].shift(1)
	# We need to define open long signal (Rsi yes>55 and RSI today>55)
	value.loc[(value['rsi'] > neutral_buy) & (value['yesterday_rsi'] < neutral_buy), 'signal_long'] = 1
	# open long signal for false positive (Rsi yes>55 and RSI today<55)
	value.loc[(value['rsi'] < neutral_buy) & (value['yesterday_rsi'] > neutral_buy), 'signal_long'] = 0
	# open long signal for overbuy (Rsi yes>70 and RSI today<70)
	value.loc[(value['rsi'] < overbuy) & (value['yesterday_rsi'] > overbuy), 'signal_long'] = 0

	"""Short Sell signal"""
	oversell = 30
	neutral_sell = 50 - neutral

	# Put the values for the short signal columns
	value['signal_short'] = np.nan
	value['yesterday_rsi'] = value["rsi"].shift(1)
	# We define the Open short signal (RSI yes>45 and RSI today<45)
	value.loc[(value['rsi'] < neutral_sell) & (value['yesterday_rsi'] > neutral_sell), 'signal_short'] = -1

	# We define the Close short signal(Rsi yes<45 and RSI today>45) False signal
	value.loc[(value['rsi'] > neutral_sell) & (value['yesterday_rsi'] < neutral_sell), 'signal_short'] = 0

	# We define the Close short signal (RSI yes<30 and RSI today>30) Oversell
	value.loc[(value['rsi'] > oversell) & (value['yesterday_rsi'] < oversell), 'signal_short'] = 0

	"""Compute the returns"""
	# Compute the percentage of the variation of the stock
	value['pct'] = value['Adj Close'].pct_change(1)

	# Calculate the position
	value['Position'] = (value['signal_short'].fillna(method='ffill') + value['signal_long'].fillna(method='ffill'))

	# Compute the returns
	value['return'] = value['pct'] * (value['Position'].shift(1))

	return value['return']


In [4]:
def beta_function(series):
	sp500 = yf.download('^GSPC')[['Adj Close']].pct_change(1)
	sp500.columns = ['SP500']

	# We concatenate the two dataframe
	value = pd.concat((series, sp500), axis=1)

	# We compute the beta
	beta = np.cov(value[[series.name, "SP500"]].dropna().values,rowvar=False)[0][1] / np.var(value["SP500"].dropna().values)
	return beta


In [5]:
def drawdown_function(series):
	cumulative_sum = series.dropna().cumsum() + 1  # multiplication coefficient
	# compute the max of the cumulative_sum
	# (1,3,1,1,7,1,2,) -> (1,3,3,3,7,7,7)
	running_max = np.maximum.accumulate(cumulative_sum)
	drawdowned = cumulative_sum / running_max - 1
	return drawdowned


In [42]:
def BackTest(serie):

  # Import the benchmarcl
  sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
  sp500.columns = ["SP500"]

  val = pd.concat((serie, sp500), axis=1).dropna()

  # Compute the drawdown
  drawdown = drawdown_function(serie)
  max_drawdown = -np.min(drawdown)

  # Put a subplots
  fig, (cum, dra) = plt.subplots(1,2, figsize=(15,6))

  # Put a Suptitle
  fig.suptitle("Backtesting", size=20)

  # Put the cumsum
  cum.plot(serie.cumsum(), color="#39B3C7")
  cum.plot(val["Adj Close"].cumsum(), color="#B85A0F")
  cum.legend(["Portfolio", "SP500"])
  # Set individual title
  cum.set_title("Cumulative Return", size=13)

  # Put the drawdown
  dra.fill_between(drawdown.index,0,drawdown, color="#C73954", alpha=0.65)

  # Set individual title
  dra.set_title("Drawdown", size=13)

  # Plot the graph
  plt.show()

  # Compute the sortino
  sortino = np.sqrt(252)*serie.mean()/serie.loc[serie<0].std()

  # Compute the beta
  beta = np.cov(val,rowvar=False)[0][1] /np.var(val["Adj Close"].dropna())

  # Compute the alpha
  alpha = 252*serie.mean() - 252*beta*serie.mean()

  # Print the statistics
  print(f"Sortino: {np.round(sortino,3)}")
  print(f"Beta: {np.round(beta,3)}")
  print(f"Alpha: {np.round(alpha,3)}")
  print(f"MaxDrawdown: {np.round(max_drawdown*100,3)} %")


In [7]:
def optimization(data):
	# Statistical approach for Parameter
	# We list for the possible values of neutral and window
	neutral_values = [i*2 for i in range(10)]
	window_values = [i*2 for i in range(1,11)]

	# Set some dataset
	start_train,end_train = "2017-01-01",'2019-01-01'
	start_test,end_test = '2019-01-01','2020-01-01'
	start_valid,end_valid = '2020-01-01','2021-01-01'

	# Initialize the list
	result = []

	for i in range(len(neutral_values)):
		for j in range(len(window_values)):
			# Compute the return
			return_train = rsi(data.loc[start_train:end_train],neutral_values[i],window_values[j])
			return_test = rsi(data.loc[start_test:end_test],neutral_values[i],window_values[j])

			# Compute the sortino
			sortino_train = np.sqrt(252) * return_train.mean() / (return_train[return_train<0].std() + 0.00001)
			sortino_test = np.sqrt(252) * return_test.mean() / (return_test[return_test<0].std() + 0.00001)

			values = [neutral_values[i],window_values[j],sortino_train,sortino_test]
			result.append(values)

	dataframe = pd.DataFrame(result,columns=["Neutral","Window","Sortino_train","Sortino_test"])
	ordered_data = dataframe.sort_values(by=["Sortino_train"],ascending=False)
	for i in range(len(dataframe)):
		# Take the best
		best = ordered_data.iloc[0+i:1+i,:]

		# Extract the sortino
		sortino_train = best["Sortino_train"].values[0]
		sortino_test = best["Sortino_test"].values[0]

		# Take best neutral and window
		best_neutral = best["Neutral"].values[0]
		best_window = best["Window"].values[0]

		# If the best is found, we stop the loop
		if sortino_test > 0.5 and sortino_train > 0.5:
			break
		else:
			best_neutral = 0
			best_window = 0
			sortino_train = 0
			sortino_test = 0
	return [best_neutral,best_window,sortino_train,sortino_test]

In [ ]:
assets = pd.read_csv('Names.csv')["Symbol"]

result = []
col = []

# Compute the best parameter for each asset
for i in tqdm(assets):
	try:
		arr = yf.download(i)
		# We find the optimal parameter
		result.append(optimization(arr))
		col.append(i)
	except:
		pass

In [16]:
# Add asset columns to each list of result
for i in range(len(result)):
	result[i].append(col[i])

# Convert list into dataframe

res = pd.DataFrame(result,columns=["Neutral","Window","Sortino_train","Sortino_test","Asset"])

# Index by asset
res = res.set_index("Asset")

# Order the dataframe in descending order
values = res.sort_values(by="Sortino_train",ascending=False)
values = values.iloc[1:,:]
values.head()






IndexError: string index out of range

In [18]:
values

,Neutral,Window,Sortino_train,Sortino_test
Asset,,,,
XRP-USD,10,2,4.218000,0.509452
BSV-USD,4,2,4.165631,2.001604
ADA-USD,8,6,4.094578,0.707319
DOGE-USD,16,2,3.582058,0.502433
MIOTA-USD,12,2,3.254906,0.529570
...,...,...,...,...
AVGO,0,0,0.000000,0.000000
PEP,0,0,0.000000,0.000000
CMCSA,0,0,0.000000,0.000000


In [39]:
# Set the borders
start_train,end_train = "2017-01-01",'2019-01-01'
start_test,end_test = '2019-01-01','2020-01-01'
start_valid,end_valid = '2020-01-01','2021-01-01'

# Create a dataframe to put the strategies in the portfolio
strategies = pd.DataFrame()
for col in values.index[:15]:
	data = yf.download(col)
	best_neutral = values.loc[col]["Neutral"]
	best_window = values.loc[col]["Window"]
	print(strategies)
	strategies[f"{col}"] = rsi(data.loc[start_train:end_train],best_neutral,best_window)
strategies.dropna().head()

[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: []
[*********************100%***********************]  1 of 1 completed
             XRP-USD
Date                
2017-11-09       NaN
2017-11-10       NaN
2017-11-11       NaN
2017-11-12       NaN
2017-11-13       NaN
...              ...
2018-12-28 -0.104613
2018-12-29 -0.025474
2018-12-30  0.000000
2018-12-31 -0.000000
2019-01-01 -0.034207

[419 rows x 1 columns]
[*********************100%***********************]  1 of 1 completed
             XRP-USD   BSV-USD
Date                          
2017-11-09       NaN       NaN
2017-11-10       NaN       NaN
2017-11-11       NaN       NaN
2017-11-12       NaN       NaN
2017-11-13       NaN       NaN
...              ...       ...
2018-12-28 -0.104613 -0.103256
2018-12-29 -0.025474 -0.044937
2018-12-30  0.000000  0.009701
2018-12-31 -0.000000  0.040696
2019-01-01 -0.034207 -0.084585

[419 rows x 2 columns]
[*********************100%*****

,XRP-USD,BSV-USD,ADA-USD,DOGE-USD,MIOTA-USD,TRX-USD,BCH-USD,XLM-USD,ETH-USD,ALGN,NTES,USDMYR=X,BIDU,NEO-USD,BTC-USD
Date,,,,,,,,,,,,,,,
2018-11-16,-0.000000,-0.312309,0.037644,-0.000000,0.023106,0.009436,0.077139,-0.000000,0.031133,0.0,0.008730,-0.001073,-0.007749,0.023338,0.012833
2018-11-19,-0.063288,0.321119,0.150753,-0.000000,0.138887,0.161600,0.127629,-0.000000,0.157522,-0.0,-0.000557,0.000119,-0.000000,-0.000000,-0.000000
2018-11-20,0.090847,0.297469,0.148435,0.067825,0.149407,0.124280,0.336107,0.139115,0.126268,0.0,-0.027778,-0.000716,0.030641,-0.000000,0.086138
2018-11-21,-0.029731,-0.001678,-0.067066,-0.086069,-0.098548,-0.064094,-0.054756,-0.054324,-0.048811,0.0,-0.029894,0.000358,-0.036455,0.000000,-0.033761
2018-11-23,-0.000000,-0.263817,0.019049,-0.000000,-0.026810,-0.000000,0.019114,0.035070,0.026913,0.0,-0.033522,0.000000,-0.000000,0.000000,0.004313


In [1]:
# Compute the portfolio
def sortino_cretiria(weights,returns):
	pf_return = returns.values.dot(weights)
	mu = np.mean(pf_return)
	sigma = np.std(pf_return[pf_return<0])
	sortino = -mu/sigma
	return sortino

num = strategies.shape[1]

# Initialize the weights
x0 = np.zeros(num)+(1/num)

# Optimization constraints
cons = ({'type':'eq','fun':lambda x:sum(abs(x))-1})

# Put th bounds
bound = [(0,1) for i in range(0,num)]

# Run the optimization of sortino
res_sortino = minimize(sortino_cretiria,x0,method="SLSQP",args=(strategies.loc[start_train:end_test].dropna()),bounds=bound,constraints=cons,options={'disp':False})


X_SR = res_sortino.x
np.round(X_SR,3)

NameError: name 'strategies' is not defined

In [46]:
sr = np.multiply(strategies.loc[start_valid:end_valid],X_SR).sum(axis=1)
sr

Series([], dtype: float64)